In [6]:
import foodcosts.core as core
import pandas as pd

In [7]:
#| export
def get_current_stock_outs():
    columns = ["product_name","code","shop_acronym","available_qty","available_qty_rounded",
           "product_min_qty","product_max_qty", "uom", "uom legible", "converted_qty", "nbr_days_since_last_sent"]
    query = """
    select
distinct on (move.product_id, move.location_dest_id) 
CASE WHEN prod_trans.value is null or prod_trans.value = '' THEN product_template.name ELSE
    CASE WHEN product_variant_combination.product_product_id is not null and product_variant_combination.product_template_attribute_value_id != comb2.product_template_attribute_value_id
        THEN prod_trans.value || ' (' || product_attribute_value.name || ')' || ' (' || val2.name || ')'
        ELSE 
        CASE WHEN product_variant_combination.product_product_id is not null
        THEN prod_trans.value || ' (' || product_attribute_value.name || ')'
        ELSE prod_trans.value END END end as name,
prod.default_code,
loc_trans.value as shop,
coalesce(location_qties.available_qty,0) as available_qty,
round(coalesce(location_qties.available_qty,0)::numeric,3) as available_qty_rounded,
orderpoint.product_min_qty,
orderpoint.product_max_qty,
uom.name as uom,
uom_shop.name as shop_uom,
round(((location_qties.available_qty / uom.factor) * uom_shop.factor)::numeric,0) as converted_qty,
DATE_PART('day', current_date - (select op.date from stock_move op where op.product_id = prod.id and op.location_id = 12 and op.location_dest_id = stock_location.id and op.product_uom_qty <> 0 and op.state = 'done' order by op.date desc limit 1)) as nb_days_since_last_sent
from stock_move move
left join product_product prod on move.product_id = prod.id
left join product_template on product_template.id = prod.product_tmpl_id

left join product_variant_combination on (product_variant_combination.product_product_id = prod.id and product_variant_combination.product_template_attribute_value_id = (SELECT max(product_template_attribute_value_id) from product_variant_combination where product_product_id = prod.id))
left join product_template_attribute_value on product_template_attribute_value.id = product_variant_combination.product_template_attribute_value_id
left join product_attribute_value on product_attribute_value.id = product_template_attribute_value.product_attribute_value_id

left join product_variant_combination comb2 on (comb2.product_product_id = prod.id and comb2.product_template_attribute_value_id = (SELECT min(product_template_attribute_value_id) from product_variant_combination where product_product_id = prod.id))
left join product_template_attribute_value value2 on value2.id = comb2.product_template_attribute_value_id
left join product_attribute_value val2 on val2.id = value2.product_attribute_value_id

left join ir_translation prod_trans on (prod_trans.res_id = product_template.id and prod_trans.lang = 'fr_BE' and prod_trans.name = 'product.template,name')
left join uom_uom uom on uom.id = product_template.uom_id
left join uom_uom uom_shop on uom_shop.id = prod.picking_uom_id
inner join stock_picking_type type on type.id = move.picking_type_id
inner join stock_warehouse_orderpoint orderpoint on orderpoint.product_id = prod.id and orderpoint.location_id = move.location_dest_id
inner join stock_location on stock_location.id = move.location_dest_id
left join ir_translation loc_trans on (loc_trans.res_id = stock_location.location_id and loc_trans.lang = 'fr_BE' and loc_trans.name = 'stock.location,name')
left join (select product_id, location_id, sum(quantity) as available_qty from stock_quant group by location_id, product_id) location_qties on (location_qties.product_id = prod.id and location_qties.location_id = move.location_dest_id)
where prod.active = True
and type.code = 'internal'
and coalesce(location_qties.available_qty,0) < orderpoint.product_min_qty 
and move.state != 'done'
and orderpoint.active = True
    """
    return pd.DataFrame(core.make_sql_query(query), columns=columns)

In [8]:
get_current_stock_outs()

,product_name,code,shop_acronym,available_qty,available_qty_rounded,product_min_qty,product_max_qty,uom,uom legible,converted_qty,nbr_days_since_last_sent
0,HACHIS PARMENTIER,9787 - 41 / FTK,WEM,0.000,0.000,0.500,1.000,KG,"Barquette 0,5Kg",None,0.0
1,POULET FERME CUIT,9520 - 01 / FTK,LAH,-7.939,-7.939,1.000,1.000,KG,Barquette 1kg,-8,4.0
2,DINDE ESCALOPE PANEE SAUCE + COULIS,9445 - 42 / FTK,LAH,2.256,2.256,3.000,3.000,KG,"Pièce 0,4 kg",6,2.0
3,DINDE ESCALOPE PANEE SAUCE + COULIS,9445 - 42 / FTK,WEM,1.200,1.200,1.500,3.000,KG,"Pièce 0,4 kg",3,2.0
4,WASABI NUTS PETIT POT,9817 - 76 / FTK,LAH,2.000,2.000,4.000,8.000,Unit(s),None,None,22.0
...,...,...,...,...,...,...,...,...,...,...,...
355,BEIRUT CHICKPEA BALLS / HOUMOUS MAISON,9587 - 80 / FTK,WOL,1.000,1.000,2.000,4.000,Unit(s),None,None,0.0
356,CHILI SIN CARNE AUX POIS CHICHE / CHAKCHOUKA A...,7742 - 92 / FTK,LAH,0.000,0.000,4.000,6.000,Unit(s),None,None,9.0
357,CHILI SIN CARNE AUX POIS CHICHE / CHAKCHOUKA A...,7742 - 92 / FTK,FOJ,0.000,0.000,4.000,6.000,Unit(s),None,None,2.0
358,CHILI SIN CARNE AUX POIS CHICHE / CHAKCHOUKA A...,7742 - 92 / FTK,CHA,0.000,0.000,1.000,2.000,Unit(s),None,None,16.0


In [ ]:
columns = ["product_name","code","shop_acronym","available_qty","available_qty_rounded",
           "product_min_qty","product_max_qty", "uom", "uom legible", "converted_qty", "nbr_days_since_last_sent"]
#coalesce(location_qties.available_qty,0) as available_qty,
# round(coalesce(location_qties.available_qty,0)::numeric,3) as available_qty_rounded,
# orderpoint.product_min_qty,
# orderpoint.product_max_qty,
